In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from datetime import datetime
import plotly.express as px

In [ ]:
_today = datetime.today().strftime('%Y-%m-%d')
def to_report(pivot_table, plot, today=_today):
    with open(_today + '_weekly_report_iris.html', 'w') as f:
        f.write(pivot_table.to_html())
        f.write(plot.to_html(full_html=False, include_plotlyjs='cdn'))
        f.close()

In [ ]:
class WeeklyReport:
    
    _today = datetime.today().strftime('%Y-%m')
    
    _month_dict = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 
             5: 'May', 6: 'June', 7: 'July', 8: 'August', 
             9: 'September', 10: 'October', 11: 'November', 12: 'December'}
    
    _month_list = list(_month_dict.values())
    
    def __init__(self):
        path = self.file_path()
        df = self.sheet_select(path)
        self.df = df
        
    def file_path(self):
        root = tk.Tk()
        root.withdraw()
        path = filedialog.askopenfilename()
        return path

    def sheet_select(self, path):
        df = pd.read_excel(path, None)
        sheet_dict = {}
        for index, sheet in enumerate(df.keys()):
            print(index + 1, sheet)
            sheet_dict.setdefault(index + 1, sheet)
        slct_sheet = eval(input('請輸入要開啟的sheet編號：\n'))
        df = df[sheet_dict[slct_sheet]]
        return df
    
    @property
    def for_iris(self):
        last_month = int(self._today.split('-')[-1]) - 1
        this_month = int(self._today.split('-')[-1])
        next_month = int(self._today.split('-')[-1]) + 1
        df = self.df[(self.df['狀態'].str.contains('出')) & (self.df['BG'] == 'RF') & (self.df['預交年份'] == int(self._today.split('-')[0])) & 
                    (self.df['預交月份'].isin([self._month_dict[last_month], self._month_dict[this_month], self._month_dict[next_month]]))]
        df['廠別'] = df['品名'].map(lambda x: 'BU2' if x.startswith('RFDP') else 'BU1')
        df['Type'] = '實績'
        df = df[df['廠別'] == 'BU1']
        df = df.groupby(['負責業務', '預交月份'])[['本國幣別NTD']].sum().reset_index()
        df['預交月份'] = pd.Categorical(df['預交月份'], ordered=True, categories=self._month_list)
        
        table = df.pivot_table(index='負責業務', values='本國幣別NTD', columns='預交月份', fill_value=0, aggfunc='sum', margins=True)
        pd.options.display.float_format = '{:,.0f}'.format # 讓pivot table裡面可以使用千分位
        return table
    
    @property
    def iris_plot(self):
        last_month = int(self._today.split('-')[-1]) - 1
        this_month = int(self._today.split('-')[-1])
        next_month = int(self._today.split('-')[-1]) + 1
        df = self.df[(self.df['狀態'].str.contains('出')) & (self.df['BG'] == 'RF') & (self.df['預交年份'] == int(self._today.split('-')[0]))]
        df['預交月份'] = pd.Categorical(df['預交月份'], ordered=True, categories=self._month_list)
        df['廠別'] = df['品名'].map(lambda x: 'BU2' if x.startswith('RFDP') else 'BU1')
        df = df[df['廠別'] == 'BU1']
        plot = df.groupby(['預交月份'])[['本國幣別NTD']].sum().reset_index()
        fig = px.histogram(plot, x='預交月份', y='本國幣別NTD', category_orders={'預交月份': self._month_list}, 
                          title='BU1 Amber team每月實績', labels=dict(本國幣別NTD='Revenue', 預交月份='Month'))
        return fig
        

In [ ]:
import plotly.express as px

In [ ]:
data = df.df

In [ ]:
wip = data[(data.BG == 'RF') & (data['狀態'].str.contains('出')) & (data['負責業務'] == '許凱智')]

In [ ]:
wip['單價'] = wip.apply(lambda x: x['單價'] / 28.5 if x['幣別'] == 'NTD' else x['單價'] / 6.5 if x['幣別'] == 'CNY' else x['單價'], axis=1)

In [ ]:
wip = wip.groupby(['品名', '單價'])[['數量', '本國幣別NTD']].sum().reset_index()

In [ ]:
med_price = round(wip['單價'].mean(), 2)
med_qty = round(wip['數量'].mean(), 2)

In [ ]:
fig = px.scatter(wip, x='數量', y='單價', size='本國幣別NTD', color='品名', title='產品象限圖')
fig.update_yaxes(tickprefix='$', showgrid=True)
fig.add_shape( # add a horizontal "target" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=0, x1=1, xref="paper", y0=med_price, y1=med_price, yref="y"
)
fig.add_shape( # add a vertical "target" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=med_qty, x1=med_qty, xref="x", y0=0, y1=1, yref="paper"
)

In [ ]:
df = WeeklyReport()

In [ ]:
class SampleTracking:
    def __init__(self):
        pass

In [ ]:
# 將ipynb轉py要寫下面的程式碼
!jupyter nbconvert --to script office_assistant.ipynb 